In [1]:
import os
import json
import pandas as pd
from pathlib import Path
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
from IPython.display import display, Markdown
import torch

# Paths
paths = {
    'root': Path.cwd().parent,
    'data': Path.cwd().parent / "data",
    "config": Path.cwd().parent / "config"
}

# Load Hugging Face credentials
with open(paths["config"] / 'credentials.json') as f:
    credentials = json.load(f)


if "HUGGINGFACE_TOKEN" in os.environ or "HUGGINGFACE_TOKEN" in credentials:
    print("Environment variable HUGGINGFACE_TOKEN set.")


# Define the file path
file_path = paths["config"] / "instructions.txt"

try:
    # Open the file and read its content
    with open(file_path, 'r') as file:
        instructions = file.read()
        print("Instructions successfully read!")
        # print(instructions)
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


# Shortlisted candidates
try:
    list_of_candidates = pd.read_parquet(paths['data'] / "processed/filtered.parquet", columns=['job_title', 'rank']).set_index("rank")['job_title'].to_list()
except Exception as e:
    print(f"Failed to load parquet file: {e}. Loading CSV instead.")
    list_of_candidates = pd.read_csv(paths['data'] / "processed/filtered.csv", index_col=['rank'], usecols=['job_title'])['job_title'].to_list()

c:\Users\Guill\Miniconda3\envs\a4_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Environment variable HUGGINGFACE_TOKEN set.
Instructions successfully read!


In [2]:
# Initialize the model and tokenizer
model_name = "microsoft/Phi-3-mini-128k-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

# Adjust model loading for GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    trust_remote_code=True,
    device_map=device,  # Use GPU if available
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.18s/it]
Device set to use cuda


In [ ]:
# Load system description and user prompt template from files
with open(paths["config"] / 'role_system_description.txt', 'r') as f:
    role_system_description = f.read()

with open(paths["config"] / 'user_prompt_template.txt', 'r') as f:
    user_prompt_template = f.read()

with open(paths["config"] / 'response_format.txt', 'r') as f:
    response_format = f.read()

with open(paths["config"] / 'search_criteria.txt', 'r') as f:
    search_criteria = f.read()


In [11]:
list_similar_roles = 'aspiring human resources'
location = "New York"

# Format the user prompt using the template and parameters
user_prompt = user_prompt_template.format(
    location=location,
    list_similar_roles=list_similar_roles,
    search_criteria=search_criteria,
    list_of_candidates="\n".join(list_of_candidates)  # Joining list elements for display
)

# Prepare messages in the format required by the model
messages = [
    {"role": "system", "content": role_system_description},
    {"role": "user", "content": user_prompt},
]

# Generate text
generation_args = {
    "max_new_tokens": 5_000,
    "return_full_text": False,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
display(Markdown(f"Generated Output:\n\n{output[0]['generated_text']}"))

Generated Output:

Based on the provided criteria, here is the evaluation and ranking of the top 5 candidates suitable for a position in Human Resources located in New York:

| **Candidate**                                                                          | **Experience & Background**                                                                                                   | **Reasoning for Choosing This Candidate**                                                                                                      |
|----------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------|
| Aspiring human resources professional an energetic and teamfocused leader              | Demonstrates a strong commitment to HR with a focus on team leadership and motivation.                                        | Strong motivation and leadership skills make this candidate a great fit with company values emphasizing energetic and team-focused personnel.   |
| Aspiring human resources professional passionate about helping to create an inclusive and engaging work environment  | Shows passion for inclusivity and engagement within HR, emphasizing important company values.                                 | Combined passion for inclusivity and foundational HR interest are aligned with cultural values.                                                 |
| Aspiring human resources manager seeking internship in human resources                 | Actively seeking opportunities to gain HR experience, indicating high motivation and willingness to learn.                    | Motivated to enter HR with a focus on gaining practical experience, demonstrating alignment with company values centering on motivated learners.|
| Experienced retail manager and aspiring human resources professional                   | Brings transferable management skills from the retail sector, suitable for HR development.                                    | Possesses transferable skills valuable for HR roles, supporting a fresh perspective into developing HR practices.                                |
| CT Bauer College of Business graduate magna cum laude and aspiring human resources professional | Strong academic background in business and interest in HR, indicating the potential for growth in the field.                   | Academic excellence with a focus on Human Resources provides a foundation for significant potential in HR.                                       |

These candidates are ranked based on demonstrated motivation, relevant experience or transferable skills, and alignment with company values. They present strong potential for contributing positively to a Human Resources role with their unique capabilities and backgrounds.